In [55]:
# !pip install osmnx

In [56]:
import osmnx as ox
import networkx as nx
import pandas as pd
from geopy.distance import geodesic

In [57]:
df = pd.read_parquet("data/valenbici_2025.parquet")

In [ ]:
# Coordenadas de inicio y destino
origen = (39.4699, -0.3763)     # Ejemplo: Valencia centro
destino = (39.4750, -0.3543)    # Otro punto de la ciudad

# Descargar grafo de Valencia (peatonal o bicicleta)
#Esto se ejecuta antes de la interacción con el usuario
G = ox.graph_from_point(origen, dist=3000, network_type='bike', simplify=True)  # 'walk', 'drive', etc.

In [59]:
def heuristica(u, v):
    # Devuelve la distancia geodésica (aproximada en metros) entre los nodos u y v
    punto_u = (G.nodes[u]['y'], G.nodes[u]['x'])
    punto_v = (G.nodes[v]['y'], G.nodes[v]['x'])
    return geodesic(punto_u, punto_v).meters

In [60]:
# Convertir coordenadas a nodos
origen_nodo = ox.nearest_nodes(G, origen[1], origen[0])
destino_nodo = ox.nearest_nodes(G, destino[1], destino[0])

# Calcular ruta más corta por longitud
ruta = nx.shortest_path_length(G, origen_nodo, destino_nodo, weight='length')

print(ruta)

2429.2414805868875


In [61]:
len(df['Direccion'].unique())

273

In [ ]:
#Se ejecuta antes
coord_vb = {(float(x.split(',')[0]), float(x.split(',')[1])) for x in df['geo_point_2d']}

In [63]:
coord_vb

{(39.43982822413676, -0.3891374575312111),
 (39.44412522887422, -0.36757337969703135),
 (39.44462122517277, -0.3751583239336074),
 (39.44477922701873, -0.37706532415787625),
 (39.44525124234686, -0.38910445245389036),
 (39.44546024514969, -0.3930154660803912),
 (39.44608224133691, -0.38029642271654757),
 (39.447703249066144, -0.38602944085901075),
 (39.447714251158565, -0.3899174536874086),
 (39.447960241518516, -0.3688183809361798),
 (39.44807022576504, -0.3331882593063431),
 (39.45004424948915, -0.3705033850186671),
 (39.45028826570709, -0.40357449843597065),
 (39.450305232524904, -0.3332722584595297),
 (39.45032925434954, -0.37886641388495107),
 (39.45039125246078, -0.3740153970137416),
 (39.450807354184086, -0.35855273823228967),
 (39.451692263769644, -0.38857844617931064),
 (39.45208226727133, -0.39301546105732643),
 (39.45218324552497, -0.3472273049060467),
 (39.45230524022502, -0.3350652633218127),
 (39.45262025913349, -0.3722843899417025),
 (39.45263726161547, -0.37871441139196

In [64]:
# Paso 1: Obtener nodos de coord_vb en bloque
xs = [coord[1] for coord in coord_vb]
ys = [coord[0] for coord in coord_vb]
nodos_vb = ox.nearest_nodes(G, xs, ys)
coord_nodo_map = dict(zip(nodos_vb, coord_vb))

# Paso 2: Evita nodo origen si está en las coords (lo tratamos aparte si hace falta)
nodos_vb_filtrados = set(nodos_vb) - {origen_nodo}

# Paso 3: Calcular rutas desde origen_nodo a todos los nodos posibles
longitudes = nx.single_source_dijkstra_path_length(G, origen_nodo, weight='length')

In [65]:
# Paso 1: Obtener nodos de coord_vb vectorizado
xs = [coord[1] for coord in coord_vb]
ys = [coord[0] for coord in coord_vb]
nodos_vb = ox.nearest_nodes(G, xs, ys)
coord_nodo_map = dict(zip(nodos_vb, coord_vb))

# Paso 2: Quitar nodo destino (a menos que queramos comprobarlo aparte)
nodos_vb_filtrados = set(nodos_vb) - {destino_nodo}

# Paso 3: Calcular rutas desde destino_nodo a todos los demás nodos
longitudes = nx.single_source_dijkstra_path_length(G, destino_nodo, weight='length')

In [66]:
mejor_coord = None
longitud_minima = float('inf')

for nodo in nodos_vb_filtrados:
    if nodo in longitudes and longitudes[nodo] < longitud_minima:
        longitud_minima = longitudes[nodo]
        mejor_coord = coord_nodo_map[nodo]

# Paso 5 (opcional): tratar el caso donde origen ya es uno de los nodos objetivo
if origen_nodo in coord_nodo_map:
    mejor_coord = coord_nodo_map[origen_nodo]
    longitud_minima = 0

print(f"Coordenada más cercana por ruta: {mejor_coord}")
print(f"Longitud de la ruta más corta: {longitud_minima:.2f} metros")


Coordenada más cercana por ruta: (39.47586032883543, -0.35596831583559707)
Longitud de la ruta más corta: 198.79 metros


In [67]:
mejor_coord = None
longitud_minima = float('inf')

for nodo in nodos_vb_filtrados:
    if nodo in longitudes and longitudes[nodo] < longitud_minima:
        longitud_minima = longitudes[nodo]
        mejor_coord = coord_nodo_map[nodo]

# Caso especial: el destino está entre las coordenadas
if destino_nodo in coord_nodo_map:
    mejor_coord = coord_nodo_map[destino_nodo]
    longitud_minima = 0
    
print(f"Coordenada de coord_vb más cercana al destino: {mejor_coord}")
print(f"Longitud de la ruta más corta: {longitud_minima:.2f} metros")
print(destino)


Coordenada de coord_vb más cercana al destino: (39.47586032883543, -0.35596831583559707)
Longitud de la ruta más corta: 198.79 metros
(39.475, -0.3543)
